In [35]:
!pip install onnx onnxruntime-gpu num2words

In [36]:
!git clone https://github.com/aifoundry-org/lerobot
!cd lerobot && pip3 install .

fatal: destination path 'lerobot' already exists and is not an empty directory.
Processing /content/lerobot
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.3.4-py3-none-any.whl size=698304 sha256=52e3555e0c6d0d023714a1c85e73dfa5b09131666a86d1a23fb39be490f0b32d
  Stored in directory: /tmp/pip-ephem-wheel-cache-aeuvb0av/wheels/09/b4/fe/75732b1d640db96ba1f856f2b7328b232a03b696a39cb59686
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.3.4
    Uninstalling lerobot-0.3.4:
      Successfully uninstalled lerobot-0.3.4


In [3]:
%pip install -q ipdb
%env PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
%pdb off

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.8 MB/s eta 0:00:00
env: PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
Automatic pdb calling has been turned OFF


# Init

In [4]:
import random
import os
import numpy as np
import torch

def set_deterministic(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False
    torch.set_num_threads(1)  # reduce CPU reduction-order nois

os.environ["PYTHONHASHSEED"] = "0"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

set_deterministic(0)


In [6]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig
from lerobot.configs.policies import PreTrainedConfig
from lerobot.policies.factory import make_policy

from lerobot.datasets.lerobot_dataset import LeRobotDatasetMetadata

config = PreTrainedConfig.from_pretrained("HuggingFaceVLA/smolvla_libero")
config.pretrained_path = "HuggingFaceVLA/smolvla_libero"
ds_meta = LeRobotDatasetMetadata("HuggingFaceVLA/libero")
policy = make_policy(cfg=config, ds_meta=ds_meta).eval()
# policy = make_policy(cfg=config).eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

info.json: 0.00B [00:00, ?B/s]

stats.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/868 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [7]:
import torch
from torch import nn
from lerobot.policies.smolvla.smolvlm_with_expert import SmolVLMWithExpertModel

from typing import Optional, List


In [8]:
class SmolVLMVision(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

    def forward(self, image: torch.Tensor):
        return self.model.embed_image(image)

In [9]:
class SmolVLMText(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

    def forward(self, tokens: torch.Tensor):
        return self.model.embed_language_tokens(tokens)

In [10]:
class SmolVLMForONNXExport(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

        self.config = SmolVLAConfig

        # Bake in the configuration for the static graph
        self.attention_mode = self.config.vlm_model_name
        self.num_vlm_layers = self.model.num_vlm_layers
        self.num_expert_layers = self.model.num_expert_layers
        self.fill_kv_cache = False
        self.use_cache = True

    def forward(
          self,
          attention_mask: torch.Tensor,
          position_ids: torch.LongTensor,
          vlm_embeds: torch.Tensor,
          expert_embeds: torch.Tensor,
          *past_key_values_flat: torch.Tensor,
          ):
          # Flattened past_key_values: one tensor for keys, one for values per layer


          dummy = (vlm_embeds + 0) * 1

          print(vlm_embeds)
          inputs_embeds = [vlm_embeds, None]

          # print(past_key_values_flat)

          past_key_values = {}
          if past_key_values_flat:
              # Logic to reconstruct the dictionary from the flat tuple of tensors
              # This assumes a consistent ordering: (key_layer0, val_layer0, key_layer1, val_layer1, ...)
              for i in range(0, len(past_key_values_flat), 2):
                  layer_idx = i // 2
                  past_key_values[layer_idx] = {
                      "key_states": past_key_values_flat[i],
                      "value_states": past_key_values_flat[i+1],
                  }
          else:
              past_key_values = None

          print("PAST_KEY_VALUES")
          print(past_key_values)


          final_embeds_list, new_past_key_values = self.model.forward(
              attention_mask=attention_mask,
              position_ids=position_ids,
              past_key_values=past_key_values,
              inputs_embeds=inputs_embeds,
              use_cache=self.use_cache,
              fill_kv_cache=self.fill_kv_cache,
          )

          # --- 3. Flatten Outputs ---
          # The output must be a flat tuple of tensors
          vlm_output_embeds = final_embeds_list[0]
          expert_output_embeds = final_embeds_list[1] # expects to be None

          present_key_values_flat = []
          if new_past_key_values:
            for i in range(self.num_vlm_layers):
                if i in new_past_key_values:
                    present_key_values_flat.append(new_past_key_values[i]["key_states"])
                    present_key_values_flat.append(new_past_key_values[i]["value_states"])

          return (vlm_output_embeds, expert_output_embeds, *present_key_values_flat)

In [11]:
class SmolVLMForONNXExportDecode(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

        self.config = SmolVLAConfig

        # Bake in the configuration for the static graph
        self.attention_mode = self.config.vlm_model_name
        self.num_vlm_layers = self.model.num_vlm_layers
        self.num_expert_layers = self.model.num_expert_layers
        self.fill_kv_cache = False
        self.use_cache = True

    def forward(
          self,
          attention_mask: torch.Tensor,
          position_ids: torch.LongTensor,
          expert_embeds: torch.Tensor,
          *past_key_values_flat: torch.Tensor,
          ):


          inputs_embeds = [None, expert_embeds]

          past_key_values = {}
          if past_key_values_flat:
              # Logic to reconstruct the dictionary from the flat tuple of tensors
              # This assumes a consistent ordering: (key_layer0, val_layer0, key_layer1, val_layer1, ...)
              for i in range(0, len(past_key_values_flat), 2):
                  layer_idx = i // 2
                  past_key_values[layer_idx] = {
                      "key_states": past_key_values_flat[i],
                      "value_states": past_key_values_flat[i+1],
                  }
          else:
              past_key_values = None


          final_embeds_list, new_past_key_values = self.model.forward(
              attention_mask=attention_mask,
              position_ids=position_ids,
              past_key_values=past_key_values,
              inputs_embeds=inputs_embeds,
              use_cache=self.use_cache,
              fill_kv_cache=self.fill_kv_cache,
          )

          # --- 3. Flatten Outputs ---
          # The output must be a flat tuple of tensors
          vlm_output_embeds = final_embeds_list[0]
          expert_output_embeds = final_embeds_list[1] # expects to be None

          present_key_values_flat = []
          if new_past_key_values:
            for i in range(self.num_vlm_layers):
                if i in new_past_key_values:
                    present_key_values_flat.append(new_past_key_values[i]["key_states"])
                    present_key_values_flat.append(new_past_key_values[i]["value_states"])

          return (vlm_output_embeds, expert_output_embeds, *present_key_values_flat)

EXPORT FUNCTION

In [12]:
import onnxruntime as ort
import numpy as np
import onnx

def export_onnx(model,
                input_data,
                model_name,
                input_names,
                do_constant_folding=False,
                device="cuda:0"):
  print(f"Exporting model {model_name}")
  model.eval()
  model.to(torch.float32).to(device)
  pt_output = model.forward(input_data).detach().cpu().numpy()

  torch.onnx.export(
    model,
    input_data,
    model_name,
    input_names=input_names,
    opset_version=17,
    do_constant_folding=do_constant_folding)

  onnx.checker.check_model(model_name)

  sess = ort.InferenceSession(model_name, providers=["CPUExecutionProvider"])
  onnx_output = sess.run(None, {input_names[0]: input_data.cpu().numpy().astype(np.float32)})[0] # only single input for now

  print("shape PT/ONNX:", pt_output.shape, onnx_output.shape)
  print("Mean:", pt_output.mean())
  print("STD", pt_output.std())
  print("max |diff|:", np.max(np.abs(pt_output-onnx_output)))

  return model_name




## Prefill model

In [45]:
onnx_model = SmolVLMForONNXExport("").to(torch.float32)
onnx_model.eval()

onnx_model.fill_kv_cache = True

onnx_model.to("cuda:0")

# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["vlm_output_embeds"]
num_layers = onnx_model.model.num_vlm_layers
for i in range(num_layers):
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size

batch_size = 1
# vlm_seq_len = 113 # for one image
# vlm_seq_len = 177 # for 2 images
vlm_seq_len = 151 # for 2 images
expert_seq_len = 16
total_seq_len = vlm_seq_len # No expert output

dummy_vlm_embeds = torch.randn(batch_size, vlm_seq_len, 960, device="cuda:0", dtype=torch.float32)
dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.float32) # input_embeds
dummy_attn_mask = torch.ones(batch_size, total_seq_len, total_seq_len, device="cuda:0", dtype=torch.bool)
dummy_pos_ids = torch.arange(total_seq_len, device="cuda:0").unsqueeze(0)


dummy_inputs = (dummy_attn_mask, dummy_pos_ids, dummy_vlm_embeds, None)

# we ant past_key_values from it, the second output

torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_prefill.onnx",
    # input_names=["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"],
    input_names=["attention_mask", "position_ids", "vlm_embeds", "expert_embeds"],
    output_names=output_names,
    opset_version=17
)


tensor([[[-1.1948,  0.2091, -0.6772,  ..., -1.2741, -0.3138, -1.4400],
         [ 0.0297,  1.4239, -1.2462,  ...,  0.9809, -0.3190, -0.3254],
         [-0.2583, -0.5678, -2.3568,  ...,  0.1349, -0.0970,  1.0323],
         ...,
         [-1.6320, -0.7847, -0.4762,  ..., -0.8695,  0.4956, -0.4694],
         [ 1.1993,  0.7047, -0.1045,  ..., -0.2392,  1.8764,  1.8129],
         [ 0.3004, -1.4063, -0.0620,  ...,  2.5351,  2.7645,  0.8326]]],
       device='cuda:0')
PAST_KEY_VALUES
None


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/linear.py:125: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:233.)
  return F.linear(input, self.weight, self.bias)
/usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/smolvlm_with_expert.py:237: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant

In [46]:
_, _, *past_key_values = onnx_model.forward(*dummy_inputs)

tensor([[[-1.1948,  0.2091, -0.6772,  ..., -1.2741, -0.3138, -1.4400],
         [ 0.0297,  1.4239, -1.2462,  ...,  0.9809, -0.3190, -0.3254],
         [-0.2583, -0.5678, -2.3568,  ...,  0.1349, -0.0970,  1.0323],
         ...,
         [-1.6320, -0.7847, -0.4762,  ..., -0.8695,  0.4956, -0.4694],
         [ 1.1993,  0.7047, -0.1045,  ..., -0.2392,  1.8764,  1.8129],
         [ 0.3004, -1.4063, -0.0620,  ...,  2.5351,  2.7645,  0.8326]]],
       device='cuda:0')
PAST_KEY_VALUES
None


In [47]:
past_key_values[0].shape

torch.Size([1, 151, 5, 64])

## Decode model

In [48]:
onnx_model = SmolVLMForONNXExportDecode("")
onnx_model.to(torch.float32).to("cuda:0")
onnx_model.eval()
onnx_model.fill_kv_cache = False
onnx_model.use_cache = True
chunk_size = 50
current_seq_len = past_key_values[0].shape[1] + chunk_size

batch_size = 1

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size
num_layers = onnx_model.model.num_vlm_layers
num_kv_heads = onnx_model.model.config.text_config.num_key_value_heads
head_dim = vlm_hidden_dim // onnx_model.model.config.text_config.num_attention_heads

dummy_expert_embeds = torch.randn(batch_size, chunk_size, expert_hidden_dim, device="cuda:0", dtype=torch.float32)
dummy_attn_mask = torch.ones(batch_size, chunk_size, current_seq_len, device="cuda:0", dtype=torch.bool)
dummy_pos_ids = torch.arange(chunk_size, device="cuda:0").unsqueeze(0)

input_names = ["attention_mask", "position_ids", "expert_embeds"]


dummy_vlm_embeds = None

dummy_inputs = (
    dummy_attn_mask,
    dummy_pos_ids,
    # dummy_vlm_embeds,
    dummy_expert_embeds,
    *past_key_values
)


for i in range(num_layers):
    input_names.append(f"past_key_{i}")
    input_names.append(f"past_value_{i}")


# Outputs include the embeddings and the *updated* KV cache
# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["expert_output_embeds"]
for i in range(num_layers):
    # The output names must be consistent for the next iteration
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

# dynamic_axes = {"vlm_embeds": {}, "expert_embeds": {}} # Add other static inputs if needed
# for i in range(num_layers):
#     # The '2' axis is the sequence length dimension in the KV cache shape
#     # (batch_size, num_heads, sequence_length, head_dim)
#     dynamic_axes[f"past_key_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"past_value_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"present_key_{i}"] = {2: "total_sequence_length"}
#     dynamic_axes[f"present_value_{i}"] = {2: "total_sequence_length"}

# we need output_embeds - first output
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_decode.onnx",
    input_names=input_names,
    output_names=output_names,
    # dynamic_axes=dynamic_axes,
    opset_version=17,
    do_constant_folding=False,
)


## Vision model

In [17]:
import onnx
from onnx import helper, TensorProto, shape_inference

dummy_image = torch.ones(1, 3, 512, 512, device="cuda:0", dtype=torch.float32)
img_model = SmolVLMVision("")
img_model.eval()
img_model.to(torch.float32).to("cuda:0")

emb = img_model.forward(dummy_image)
print(emb.shape)

torch.onnx.export(
    img_model,
    dummy_image,
    "smolvlm_vision.onnx",
    input_names=["image"],
    output_names=["output_embeds"],
    opset_version=17,
    do_constant_folding=False,
)


GATHERS = {"Gather", "GatherND", "GatherElements"}
INT_OK = {TensorProto.INT64, TensorProto.INT32}

def _type_map(graph):
    t = {}
    for vi in list(graph.input) + list(graph.value_info) + list(graph.output):
        tt = vi.type.tensor_type
        if tt and tt.elem_type:
            t[vi.name] = tt.elem_type
    for init in graph.initializer:
        t[init.name] = init.data_type
    return t

def patch_gather_indices_once(onnx_in, onnx_out, model_input_name="pixel_values"):
    m = onnx.load(onnx_in)
    try:
        m = shape_inference.infer_shapes(m)
    except Exception:
        pass

    g = m.graph
    vtype = _type_map(g)

    # map original indices name -> casted name (to reuse and avoid SSA violation)
    casted = {}

    # Build consumer map to find first use position
    consumers = {}
    for i, n in enumerate(g.node):
        for inp in n.input:
            consumers.setdefault(inp, []).append(i)

    def ensure_cast(idx_name: str) -> str:
        # Don’t touch the real image input even if (pathologically) used as indices
        if idx_name == model_input_name:
            raise RuntimeError(
                f"Image tensor '{idx_name}' is wired as Gather indices. "
                "This path needs GridSample-style rewrite, not a cast."
            )
        if idx_name in casted:
            return casted[idx_name]

        # If already integer typed, no cast needed
        et = vtype.get(idx_name, None)
        if et in INT_OK:
            casted[idx_name] = idx_name
            return idx_name

        # Create a unique cast output name and insert once before first consumer
        suffix = "_idx_i64"
        out_name = idx_name + suffix
        # Ensure uniqueness if something already has that name
        cnt = 1
        all_names = {x.name for x in g.node} | {x.name for x in g.input} | {x.name for x in g.output}
        all_val_names = set()
        for vi in list(g.input)+list(g.value_info)+list(g.output):
            all_val_names.add(vi.name)
        for init in g.initializer:
            all_val_names.add(init.name)
        while out_name in all_val_names:
            cnt += 1
            out_name = f"{idx_name}{suffix}_{cnt}"

        # Insert cast node before first consumer of idx_name
        first_pos = min(consumers.get(idx_name, [0]))
        cast_node = helper.make_node(
            "Cast", inputs=[idx_name], outputs=[out_name],
            to=TensorProto.INT64, name=f"{idx_name}_to_i64"
        )
        g.node.insert(first_pos, cast_node)

        # Update maps so subsequent inspections see the casted name
        casted[idx_name] = out_name
        vtype[out_name] = TensorProto.INT64
        return out_name

    # Rewrite Gather* nodes to use the single cast per indices tensor
    for n in g.node:
        if n.op_type in GATHERS and len(n.input) > 1:
            n.input[1] = ensure_cast(n.input[1])

    onnx.checker.check_model(m)
    onnx.save(m, onnx_out)

patch_gather_indices_once("smolvlm_vision.onnx", "smolvlm_vision.onnx")

torch.Size([1, 64, 960])


/usr/local/lib/python3.12/dist-packages/transformers/models/smolvlm/modeling_smolvlm.py:146: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for batch_idx, p_attn_mask in enumerate(patch_attention_mask):
/usr/local/lib/python3.12/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask
/usr/local/lib/python3.12/dist-packages/transformers/models/smolvlm/modeling_smolvlm.py:459: TracerWarning: Converting a tensor to a Python integer might cause the 

In [18]:
import onnxruntime as ort
import numpy as np

# providers = ["CUDAExecutionProvider"] if torch.cuda.is_available() else ["CPUExecutionProvider"]
providers = ["CPUExecutionProvider"]

vision_session = ort.InferenceSession("smolvlm_vision.onnx", providers=providers)

image = np.ones((1, 3, 512, 512), dtype=np.float32)

input_name = vision_session.get_inputs()[0].name

onnx_outputs = vision_session.run(None, {input_name: image})

## Text model

In [19]:
text_model = SmolVLMText("").to(torch.float32).cuda()
processor = text_model.model.processor
tokens = processor(text="Hello this is a test text prompt", return_tensors="pt")
print(f"Text tokens = {tokens}")

embs = text_model.forward(tokens["input_ids"].cuda())
print(f"Shape of text embeddings = {emb.shape}")

torch.onnx.export(
    text_model,
    tokens["input_ids"].cuda(),
    "smolvlm_text.onnx",
    input_names=["tokens"],
    output_names=["output_embeds"],
    dynamic_axes={"tokens": {0: "batch", 1: "sequence_length"}, "output_embeds": {0: "batch", 1: "sequence_length"}},
    opset_version=17,
)

Text tokens = {'input_ids': tensor([[19556,   451,   314,   253,  1028,  1694,  6011]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Shape of text embeddings = torch.Size([1, 64, 960])


# Projectors

---



In [20]:
import onnxruntime as ort
import numpy as np

## State projector

In [21]:
state_projector = policy.model.state_proj.to(torch.float32).to("cuda:0")
input = torch.randn(1, policy.config.max_state_dim, device="cuda:0")

export_onnx(state_projector, input, "state_projector.onnx", ["state"])

Exporting model state_projector.onnx
shape PT/ONNX: (1, 960) (1, 960)
Mean: -0.002484973
STD 0.5369883
max |diff|: 3.5762787e-07


'state_projector.onnx'

## Action projectors

In [22]:
action_in_projector =  policy.model.action_in_proj.to(torch.float32).to("cuda:0")
input = torch.ones(1, 50, policy.config.max_action_dim, device="cpu") # num of chunks = 50

export_onnx(action_in_projector, input, "action_in_projector.onnx", ["action"], device="cpu")

Exporting model action_in_projector.onnx
shape PT/ONNX: (1, 50, 480) (1, 50, 480)
Mean: 0.014834899
STD 0.7499991
max |diff|: 0.0


'action_in_projector.onnx'

In [23]:
action_out_projector = policy.model.action_out_proj.to(torch.float32).to("cuda:0")
input = torch.randn(1, 50, policy.model.vlm_with_expert.expert_hidden_size, device="cpu")

export_onnx(action_out_projector, input, "action_out_projector.onnx", ["action"], device="cpu")

Exporting model action_out_projector.onnx
shape PT/ONNX: (1, 50, 32) (1, 50, 32)
Mean: 0.023492547
STD 0.5753485
max |diff|: 8.34465e-07


'action_out_projector.onnx'

## Time projectors

In [24]:
time_out_projector = policy.model.action_time_mlp_out.to(torch.float32).to("cuda:0")
input = torch.ones(1, 50, policy.model.vlm_with_expert.expert_hidden_size, device="cpu") # chunks = 50

export_onnx(time_out_projector, input, "time_out_projector.onnx", ["time"], device="cpu")

Exporting model time_out_projector.onnx
shape PT/ONNX: (1, 50, 480) (1, 50, 480)
Mean: -0.025680223
STD 0.7256477
max |diff|: 5.364418e-07


'time_out_projector.onnx'

In [25]:
time_in_projector = policy.model.action_time_mlp_in.to(torch.float32).to("cpu")
input = torch.ones(1, 50, policy.model.vlm_with_expert.expert_hidden_size * 2, device="cpu") # chunks = 50

export_onnx(time_in_projector, input, "time_in_projector.onnx", ["time"], device="cpu")

Exporting model time_in_projector.onnx
shape PT/ONNX: (1, 50, 480) (1, 50, 480)
Mean: 0.52598715
STD 0.900624
max |diff|: 8.34465e-07


'time_in_projector.onnx'

# MISC

In [26]:
import os, onnx

In [27]:
out_name = None
src_name = "smolvlm_expert_prefill.onnx"
out_dir = "splited"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )

In [28]:
out_name = None
src_name = "smolvlm_expert_decode.onnx"
out_dir = "splited_decoder"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )

In [29]:
out_name = None
src_name = "smolvlm_vision.onnx"
out_dir = "splited_vision"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
)

In [30]:
!cp *.onnx "/content/drive/MyDrive/Colab Notebooks/MODELS/libero"